In [ ]:
## installing Google Translator package
#!pip install -q googletrans
#!pip install git+https://github.com/BoseCorp/py-googletrans.git --upgrade

!pip install googletrans==3.1.0a0
from googletrans import Translator

#!pip install translators --upgrade
#import translators as ts

In [ ]:
import pandas as pd
import time
from dask import bag, diagnostics

In [ ]:
df_train = pd.read_csv("/kaggle/input/text-entailment-snli/train.csv")
df_test = pd.read_csv("/kaggle/input/text-entailment-snli/test.csv")

In [ ]:
def translation(sequence):
    
    #instantiate translator
    translator = Translator()
    
    try:
        translated = translator.translate(sequence, dest = "ta").text
    except Exception as e:
        print(f"Error during translation: {e}")
        translated = sequence
    
    return translated

In [ ]:
def  translate_to_Tamil(df, category, setnum):    
    prem_bag_en = bag.from_sequence(df['premise'].tolist()).map(lambda x: translation(x))
    hyp_bag_en =  bag.from_sequence(df['hypothesis'].tolist()).map(lambda x: translation(x))
    
    with diagnostics.ProgressBar():
        prems = prem_bag_en.compute()
        hyps = hyp_bag_en.compute()
    
    print(f"\nprems: {prems}")
    print(f"\nhyps: {hyps}")
    
    df_buffer = pd.DataFrame(list(zip(prems, hyps)), columns = ['premise', 'hypothesis'])
    #df_buffer[['premise', 'hypothesis']] = list(zip(prems, hyps))
    df_buffer[['lang_abv', 'language']] = ['ta', 'Tamil']
    
    filename = category+"_ta_"+set_num+".csv"
    df_buffer.to_csv(filename, index = False)
    
    print(df_buffer)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
set_num = "35"
df_sample = df_train[340:350]
print(df_sample)

In [ ]:
translate_to_Tamil(df_sample, "train", set_num)